<a href="https://colab.research.google.com/github/DeeTV/HackCBRE2023/blob/main/HackathonCBRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

In [ ]:
data=pd.read_csv('Maria Williams.csv')                                   #data is the propert_insight data from location and client

In [ ]:
from fuzzywuzzy import fuzz
role_data=pd.read_csv('MW3.csv')         #role data is from the role_duties table
user_data2=pd.read_csv('MW2.csv')
duties=role_data.iloc[0][0]
business_line=user_data2.iloc[0][2]   #from user data
relevant_insights=role_data.iloc[0][1]
# Function to find the most relevant match among relevant_insights, duties, and business_line
def find_most_relevant_match(relevant_insights, duties, business_line, driver, insight1, insight2):
    # Initialize variables to track the most relevant match and its score
    most_relevant_match = None
    highest_score = 0

    # Create lists of all texts to match
    texts_to_match = [driver, insight1, insight2]
    insights=relevant_insights.split(",")
    insights.append(duties)
    insights.append(business_line)

    # Iterate through relevant insights, duties, and business_line, and texts to match
    for text_to_match in texts_to_match:
        for text in insights:
            # Calculate a relevance score (you can customize this scoring)
            score = fuzz.partial_ratio(text, text_to_match)

            # Update if the current text has a higher score
            if score > highest_score:
                most_relevant_match = text
                highest_score = score

    return most_relevant_match, highest_score

most_relevant_matches = []
relevance_scores = []

# Iterate through the rows in your data
for index, row in data.iterrows():
    text_to_match_driver = row["Driver"]
    text_to_match_insight1 = row["Insight_1"]
    text_to_match_insight2 = row["Insight_2"]

    # Find the most relevant match for the current row
    most_relevant_match, highest_score = find_most_relevant_match(relevant_insights, duties, business_line, text_to_match_driver, text_to_match_insight1, text_to_match_insight2)

    most_relevant_matches.append(most_relevant_match)
    relevance_scores.append(highest_score)

# Add the results as new columns in your dataset
data["Most Relevant Insight"] = most_relevant_matches
data["Relevance Score"] = relevance_scores


Responsible for all engineering and technical disciplines.
Energy Management
Energy Efficiency, Equipment Monitoring


In [ ]:
data

,Insight_1,Insight_2,Driver,Account,Criticality,City,Most Relevant Insight,Relevance Score
0,Compliance,Compliance Score,Multiple non-compliance incidents,NexaRetail,Low,San Francisco,Energy Efficiency,53
1,Energy Usage,Energy Efficiency,High energy usage during peak hours,NexaRetail,Low,San Francisco,Energy Efficiency,100
2,Work Orders,Reactive,Completion Rates below 50%,NexaRetail,High,San Francisco,Responsible for all engineering and technical ...,38
3,Energy Usage,Off-Hours,High energy usage during peak hours,NexaRetail,High,San Francisco,Energy Management,75
4,Work Orders,Preventative,Completion Rates below 50%,NexaRetail,High,San Francisco,Equipment Monitoring,42
5,Budgets,Budget Compliance,Energy Costs above 15% mean,NexaRetail,High,San Francisco,Energy Management,47
6,Budgets,Budget Compliance,Energy Costs above 15% mean,NexaRetail,Medium,San Francisco,Energy Management,47
7,Compliance,Regulatory Updates,Multiple non-compliance incidents,NexaRetail,Medium,San Francisco,Energy Efficiency,53
8,Compliance,Non-Compliance Incidents,Upcoming regulatory changes,NexaRetail,Medium,San Francisco,Responsible for all engineering and technical ...,50
9,Leases,Expiring,High number of expiring leases,NexaRetail,Critical,San Francisco,Equipment Monitoring,62


In [ ]:
# Filter the dataset based on the Relevance Score
filtered_data = data[data["Relevance Score"] > 80]

# Now, `filtered_data` contains only the rows with a Relevance Score greater than 80
filtered_data

,Insight_1,Insight_2,Driver,Account,Criticality,City,Most Relevant Insight,Relevance Score
1,Energy Usage,Energy Efficiency,High energy usage during peak hours,NexaRetail,Low,San Francisco,Energy Efficiency,100


In [ ]:
from pandas.core.missing import F
# Define a mapping from text labels to numerical labels
label_mapping = {
    'High': 2,
    'Critical': 3,
    'Medium': 1,
    'Low': 0
}

# Use the map function to apply the mapping to the "Criticality" column
filtered_data['Criticality'] = filtered_data['Criticality'].map(label_mapping)


<ipython-input-181-9dc466402672>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Criticality'] = filtered_data['Criticality'].map(label_mapping)


In [ ]:
# Sort the DataFrame based on the "Criticality_Label" column in descending order
sorted_data = filtered_data.sort_values(by='Criticality', ascending=False)

# Now, the DataFrame is sorted by criticality in descending order
sorted_data=sorted_data.drop('Relevance Score',axis=1)
sorted_data

,Insight_1,Insight_2,Driver,Account,Criticality,City,Most Relevant Insight
1,Energy Usage,Energy Efficiency,High energy usage during peak hours,NexaRetail,0,San Francisco,Energy Efficiency


In [ ]:
merged_data = sorted_data.merge(user_data2, left_on="Most Relevant Insight", right_on="Insight", how="left")

# Handle missing values in the "Total_UA" column (fill with 0)
merged_data["Total_UA"].fillna(0, inplace=True)

In [ ]:
merged_data['rank']=merged_data['Criticality']+merged_data['Total_UA']
merged_data=merged_data.sort_values(by='rank',ascending=False)
final_insight=merged_data.drop(['Name','City','Account','Insight','Total_UA'],axis=1)

In [ ]:
final_insight.to_csv('output.csv')

,Insight_1,Insight_2,Driver,Criticality,Most Relevant Insight,Business_Line,rank
0,Energy Usage,Energy Efficiency,High energy usage during peak hours,0,Energy Efficiency,Energy Management,95
